In [22]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from scipy.stats.stats import pearsonr
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

# Project 2 Task 5

In [79]:
data = pd.read_excel("18EA1-5 database cleaned HO only-version2.xlsx")

### Data Analysis

In [2]:
data.head(5)

,Cow,Farm,System,Parity,ParityCategory,BCS,DIM,DIC,Pregnant,FatPEBV,...,"10,12/preformed",20:2n6/preformed,22/preformed,20:3n6/preformed,20:4n6/preformed,20:5n3/preformed,MFD,(C13 - C11)/OBC,(C14 - C12)/DN,Trans as % of preformed
0,43,1,CM,5,3plus,3.250,200.0,0,N,-0.200,...,0.00000,0.000726,0.000663,0.004169,0.003576,0.000746,No,0.017162,0.305421,0.075902
1,51,1,CM,5,3plus,2.750,72.0,0,U,0.070,...,0.00162,0.000000,0.000624,0.001575,0.002038,0.000000,Yes,0.022726,0.281875,0.197800
2,405,1,CM,5,3plus,NaN,212.0,0,N,NaN,...,0.00000,0.001104,0.001000,0.003100,0.004451,0.000705,No,0.010826,0.240153,0.084820
3,408,1,CM,4,3plus,2.750,211.0,0,U,-0.070,...,0.00000,0.001005,0.000442,0.002461,0.004562,0.000727,Yes,0.013623,0.296892,0.105783
4,423,1,CM,5,3plus,2.875,228.0,0,N,-0.215,...,0.00000,0.000849,0.000583,0.003857,0.003656,0.000480,Yes,0.011611,0.297311,0.098821


In [3]:
data.System.describe()

count     1733
unique       2
top         CM
freq       976
Name: System, dtype: object

In [8]:
##Choosing to analyze only pregnant cows

data[data.Pregnant == "P"].shape
pregnant = data[data.Pregnant == "P"]

print(pregnant.shape)

(919, 130)


In [9]:
##waiting BCS - can adjust this

def helper(row):
    #return row.BCS
    if row.BCS > 3:
        return (5 - row.BCS)
    elif row.BCS <= 3:
        return (row.BCS)
    

def helper_mfd(row):
    if row.MFD == "Yes":
        return 1
    return 0



pregnant["BCS_adjusted"] = pregnant.apply(helper,axis=1)
pregnant.MFD = pregnant.apply(helper_mfd,axis=1)
#print(list(pregnant)[102:])data = pd.read_excel("18EA1-5 database cleaned HO only-version2.xlsx")

In [10]:
testing = pregnant[list(pregnant)[102:] + ["AvgMilk","Fat%","FatY"]]
testing = testing.dropna()

print(testing.shape)

for name in list(testing):
    print(pearsonr(testing[name],testing["AvgMilk"])[0],name)
    pass


X_train, X_test, y_train, y_test = train_test_split(testing.drop(["AvgMilk","Fat%","FatY"],axis=1), testing["AvgMilk"],test_size=0.25)
lasso = Lasso(alpha=0)
print("Cross val ", cross_val_score(lasso, X_train, y_train, cv=10).mean())
lasso.fit(X_train,y_train)
print(lasso.score(X_test, y_test))



(657, 32)
-0.1398229246987522 18:0/preformed
0.08540313412325912 t4/preformed
0.07307573484244075 t5/preformed
0.1670758495957064 t6-8/preformed
0.13438111675940273 t9/preformed
0.15183226528148383 t10/preformed
-0.06440307616593599 t11/preformed
0.3790039620258264 t12/preformed
0.08125031620986356 181c9/preformed
-0.08027937853402746 t15/preformed
0.45645255684116737 181c11/preformed
0.03894879525762649 181c12/preformed
-0.11019549781054284 18:2:9:12/preformed
-0.27130074548723493 20/preformed
-0.08345756894280736 18:3,6,9,12/preformed
0.20707963526380882 20:1/preformed
-0.004856784484236916 18:3,9,12,15/preformed
0.028630401784711202 911/preformed
0.09466022285641593 10,12/preformed
-0.22236148485564056 20:2n6/preformed
-0.2514613742464265 22/preformed
-0.2004381756855989 20:3n6/preformed
-0.2984929896186015 20:4n6/preformed
0.1959296963297143 20:5n3/preformed
0.21720769514035732 MFD
0.3391366291177437 (C13 - C11)/OBC
0.356271304633944 (C14 - C12)/DN
0.16613952629883952 Trans as % of

In [11]:
milkAvg = pregnant[["AvgMilk","DIM","DIC","t12/preformed","181c11/preformed","20/preformed","20:1/preformed",'MFD',"20:2n6/preformed","22/preformed","20:3n6/preformed","20:4n6/preformed","BCS_adjusted"]]
milkAvg = milkAvg.dropna()
print(milkAvg.shape)


X_train, X_test, y_train, y_test = train_test_split(milkAvg.drop("AvgMilk",axis=1), milkAvg["AvgMilk"],test_size=0.3)
print("MILK")
lasso = Lasso(alpha=0)
lasso.fit(X_train,y_train)
print(lasso.score(X_test, y_test))

#Random Forest for Model 1
randomf = RandomForestRegressor(n_estimators = 100)

randomf.fit(X_train,y_train)
print(randomf.score(X_test,y_test))



#################################################
#################################################
#################################################
#################################################
#################################################
#################################################
print("FATTYP")
fatP = pregnant[["DIM","DIC","FatPEBV","18:0/preformed","t6-8/preformed","t9/preformed","t10/preformed","t12/preformed","181c11/preformed","20/preformed","18:3,6,9,12/preformed","911/preformed","10,12/preformed","20:2n6/preformed","22/preformed","20:3n6/preformed","20:4n6/preformed","Trans as % of preformed", "Fat%"]]
fatP = fatP.dropna()
print(fatP.shape)

X_train, X_test, y_train, y_test = train_test_split(fatP.drop("Fat%",axis=1), fatP["Fat%"],test_size=0.3)

lasso = Lasso(alpha=0)
lasso.fit(X_train,y_train)
print(lasso.score(X_test, y_test))

#Random Forest for Model 1
randomf = RandomForestRegressor(n_estimators = 100)

randomf.fit(X_train,y_train)
print(randomf.score(X_test,y_test))

broken = pregnant[["DIM","DIC","FatPEBV","Parity","BCS_adjusted","AvgRum", "AvgMilk","Fat%","FatY",]]
broken = broken.dropna()
inputs = broken[["DIM","DIC","FatPEBV","Parity","BCS_adjusted","AvgRum"]]
output = broken[["AvgMilk"]]

(661, 13)
MILK
0.4491885844397981
0.5092548608683317
FATTYP
(846, 19)
0.4335867969763677
0.412188777489583


In [12]:
milkAvg = pregnant[[]]

In [13]:
output_name = "Fat%"
for name in list(broken):
    """plt.figure(figsize=(13,7))
    plt.scatter(fatP[name], fatP[output_name])
    plt.title(name + ' vs ' + output_name)
    plt.xlabel(name)
    plt.ylabel(output_name)"""
    print(pearsonr(broken[name], broken["AvgMilk"])[0],name)


-0.4724998276277692 DIM
-0.32392081229291936 DIC
-0.15649752371197642 FatPEBV
0.15065842514765776 Parity
0.3205551223408604 BCS_adjusted
0.16981157829802587 AvgRum
1.0 AvgMilk
-0.2554153789350104 Fat%
0.6785422021689433 FatY


In [14]:
X_train, X_test, y_train, y_test = train_test_split(inputs, broken["Fat%"], test_size=0.3)

In [17]:
lasso = Lasso(alpha=0)
lasso.fit(X_train,y_train)
print(lasso.score(X_test, y_test))





0.11956669771620299


In [113]:
data.isnull().sum().sort_values().index

Index(['Cow', '16/16C', 'i16/OBC', '15/OBC', '14:1c9/DN', 'a15/OBC', '14/DN',
       '13/OBC', 'C10:1c9/DN', 'C10/DN',
       ...
       'C18:3c6c9c12', 'C18:3c9c12c15', 'FatPEBV', 'C18:1t5', 'iC13:0',
       'i130/OBC', 'C18:1t4', 'C20:1c11', 'BCS', 'C20:2n6'],
      dtype='object', length=130)

In [118]:
null_summary = data.isnull().sum().sort_values()

In [122]:
null_summary.values

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   2,
         2,   7,   8,   9,  11,  13,  14,  14,  16,  16,  22,  25,  32,
        56, 108, 108, 114, 116, 118, 170, 171, 171, 177, 273, 452, 959])

In [124]:
for i, j in zip(null_summary.index, null_summary.values):
    print(i, j)

Cow 0
16/16C 0
i16/OBC 0
15/OBC 0
14:1c9/DN 0
a15/OBC 0
14/DN 0
13/OBC 0
C10:1c9/DN 0
C10/DN 0
C8/DN 0
16c9/16C 0
C6/DN 0
C22:5n3 0
C22:4n6 0
C24:1n9 0
C24:0 0
C20:5n3 0
CLAt10c12 0
C18:1c12 0
C18:1c11 0
C18:1c9 0
C18:1t10 0
C4/DN 0
17:1c10/OBC 0
18:0/preformed 0
t4/preformed 0
MFD 0
20:5n3/preformed 0
20:4n6/preformed 0
20:3n6/preformed 0
22/preformed 0
20:2n6/preformed 0
10,12/preformed 0
911/preformed 0
18:3,9,12,15/preformed 0
20:1/preformed 0
18:3,6,9,12/preformed 0
20/preformed 0
18:2:9:12/preformed 0
181c12/preformed 0
181c11/preformed 0
t15/preformed 0
181c9/preformed 0
t12/preformed 0
t11/preformed 0
t10/preformed 0
t9/preformed 0
t6-8/preformed 0
t5/preformed 0
C18:1t9 0
C18:0 0
Trans as % of preformed 0
C10:1c9 0
aC15:0 0
OBCyield 0
C14:0 0
C13:0 0
DN 0
OBC 0
C17:1c10 0
16C 0
C14:1c9 0
C10:0 0
C6:0 0
C4:0 0
AvgRum 0
18Cplusyield 0
18C 0
18Cplus 0
18Cyield 0
16Cyield 0
C8:0 0
C15:0 0
0.4 FCM 0
SCCS 0
Farm 0
System 0
Parity 0
C16:1c9 0
ParityCategory 0
C16:0 0
DIC 0
ECM 0
Preg

In [135]:
filtered_data = data.select_dtypes(include="float")

In [136]:
numerical_features = filtered_data.columns
categorical_features = data.columns.difference(filtered_data.columns)

In [140]:
targets = ["AvgMilk", "Fat%", "FatY", "Pro%", "ProY"]

In [153]:
X = filtered_data.drop(targets + ["BCS", "C20:2n6"], axis=1).dropna()
y = data.loc[X.index, targets].dropna()
X = X.loc[y.index, :]

In [198]:
print(X.head())
print(y.head())

     DIM  FatPEBV  SCCS   ECM  0.4 FCM     DN       OBC    16C    18C  \
3  211.0   -0.070   5.1  96.0    86.41  24.74  4.272894  36.65  32.40   
4  228.0   -0.215   1.5  87.0    81.13  23.12  3.329128  32.65  39.15   
5  140.0    0.035   1.0  85.0    77.44  28.72  3.166403  30.98  35.25   
6   35.0    0.145   3.5  99.0    88.07  23.23  3.345810  32.25  37.91   
7   67.0   -0.075   1.0  95.0    90.33  24.44  2.911895  32.17  38.71   

   18Cplus           ...             911/preformed  10,12/preformed  \
3    32.85           ...                  0.025479              0.0   
4    39.72           ...                  0.019726              0.0   
5    35.84           ...                  0.014311              0.0   
6    38.41           ...                  0.020093              0.0   
7    39.18           ...                  0.017931              0.0   

   20:2n6/preformed  22/preformed  20:3n6/preformed  20:4n6/preformed  \
3          0.001005      0.000442          0.002461          

In [185]:
pca = PCA(n_components=115)
pca.fit(X, y["AvgMilk"])
print(pca.explained_variance_ratio_)  
print(pca.singular_values_) 

[9.12970590e-01 5.41398360e-02 2.81812590e-02 4.23315307e-03
 3.17221866e-04 1.04856566e-04 3.37852606e-05 9.17645160e-06
 3.06981736e-06 2.28280724e-06 1.79754699e-06 8.83225043e-07
 6.46079343e-07 3.56652563e-07 3.18712608e-07 2.02603777e-07
 1.75246241e-07 7.27161304e-08 6.69674179e-08 5.23718398e-08
 4.16886064e-08 3.44461978e-08 2.85432564e-08 1.46252783e-08
 1.31567192e-08 1.19436602e-08 1.01156479e-08 7.05144199e-09
 5.00635205e-09 3.92325211e-09 3.44348877e-09 3.06167917e-09
 2.43487501e-09 2.36943674e-09 2.27135323e-09 1.76785063e-09
 1.53608531e-09 1.37761922e-09 1.15268439e-09 1.00497087e-09
 8.22753987e-10 7.18816744e-10 6.54363623e-10 5.94824968e-10
 4.95848585e-10 4.79463042e-10 3.54333889e-10 3.28957448e-10
 2.92925495e-10 2.17874801e-10 1.86234201e-10 1.72035159e-10
 1.18519046e-10 1.18118741e-10 9.70962495e-11 8.93400272e-11
 7.91509183e-11 5.82358635e-11 5.07681735e-11 4.80910223e-11
 3.79116828e-11 3.72377152e-11 3.02416585e-11 2.53612166e-11
 2.42160477e-11 2.167645

In [199]:
new_X = pca.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y["FatY"],test_size = .2)

In [200]:
lasso = Lasso(alpha=0)
lasso.fit(X_train, y_train)
print(lasso.score(X_test, y_test))

0.9999477690441573


In [183]:
data.shape

(1733, 130)

In [194]:
list(X)

['DIM',
 'FatPEBV',
 'SCCS',
 'ECM',
 '0.4 FCM',
 'DN',
 'OBC',
 '16C',
 '18C',
 '18Cplus',
 'DNyield',
 'OBCyield',
 '16Cyield',
 '18Cyield',
 '18Cplusyield',
 'AvgRum',
 'C4:0',
 'C6:0',
 'C8:0',
 'C10:0',
 'C10:1c9',
 'C11:0',
 'C12:0',
 'iC13:0',
 'aC13:0',
 'C13:0',
 'iC14:0',
 'C14:0',
 'iC15:0',
 'aC15:0',
 'C14:1c9',
 'C15:0',
 'iC16:0',
 'C16:0',
 'iC17:0',
 'C16:1c9',
 'aC17:0',
 'C17:0',
 'C17:1c10',
 'C18:0',
 'C18:1t4',
 'C18:1t5',
 'C18:1t6-8',
 'C18:1t9',
 'C18:1t10',
 'C18:1t11',
 'C18:1t12',
 'C18:1c9',
 'C18:1t15',
 'C18:1c11',
 'C18:1c12',
 'C18:2c9c12',
 'C20:0',
 'C18:3c6c9c12',
 'C20:1c11',
 'C18:3c9c12c15',
 'CLAc9t11',
 'CLAt10c12',
 'C22:0',
 'C20:3n6',
 'C20:4n6',
 'C20:5n3',
 'C24:0',
 'C24:1n9',
 'C22:4n6',
 'C22:5n3',
 'C4/DN',
 'C6/DN',
 'C8/DN',
 'C10/DN',
 'C10:1c9/DN',
 'C11/OBC',
 'C12/DN',
 'i130/OBC',
 'a130/OBC',
 '13/OBC',
 'i14/OBC',
 '14/DN',
 'i15/OBC',
 'a15/OBC',
 '14:1c9/DN',
 '15/OBC',
 'i16/OBC',
 '16/16C',
 '16c9/16C',
 'a17/OBC',
 '17/OBC